# Introduction

## Objectif du projet

L'objectif du projet est de cataloguer des produits selon un code type désignant le type du produit. La prédiction du type doit se faire à partir de données textuelles (désignation et description du produit) ainsi que de données visuelles (image du produit).

## Contexte

Ce projet s’inscrit dans le challenge Rakuten France Multimodal Product Data Classification, les données et leur description sont disponibles à l’adresse : https://challengedata.ens.fr/challenges/35

* Les données textuelles : ~60 mb
* Données images : ~2.2 gb
* 99k données avec plus de 1000 classes.

## Fichiers fournis

* X_train.csv: Contient les variables explicatives destinées à l’entraînement des modèles.
  * index (nb entier): Index du produit.
  * designation (object: string): Designation courte du produit
  * description (object: string, optionnel): Description du produit. Ce champ est optionnel. Tous les produits n'ont pas de description
  * productid (int64): L'id du produit
  * imageid (int64): L'id de l'image du produit

* images.zip: Une fois extrait, un dossier contenant les images des produits. La nomenclature utilisée permet de faire la jonction avec les produits. Chaque fichier d'image se présente sous la forme: `image_<imageid>_product_<productid>.jpg`. Les images sont répartis en deux sous-dossiers:
  * image_train: Les images correspondants à X_train.csv
  * image_test: Les images correspondants à X_test.csv

* Y_train.csv: Contient la variable cible à prédire. A savoir le type du produit.
  * index (nb entier): Index du produit. Permet de faire la jonction avec X_train.csv
  * prdtypecode (nb entier): Le type du produit

* X_test.csv: Contient les variables explicatives destinées à l'évaluation des modèles. Sa structure est identique à celle de X_train.csv. Ce fichier étant fourni dans le contexte du challenge Rakuten, nous n'avons pas obtenu de fichier Y_test.csv qui nous permettrait de comparer nos performances à celles des participants au challenge. Il est probable qu'on doive se contenter de scinder le dataset d’entraînement en une partie entraînement et une partie test.

> **Note**: Nous avons légèrement modifié les entêtes des fichiers d'origine en y ajoutant le titre de colonne index pour la première colonne


## Métrique utilisée pour l'évaluation de la performance

La métrique **weighted-F1 score** a été choisie dans le cadre du challenge.

## Intérêts du projet

Étant réalisé dans le cadre de la formation datascientest, ça va aussi être l'opportunité pour nous de découvrir et mettre en applications des techniques de machine learning avancées telle que:

* Computer vision
  * réseaux de neurones convolutifs
* NLP
* Modèles multimodaux
* deep learning

Bien que ce projet utilise le dataset d'un site de vente, il est assez générique de par la nature de son sujet qui pourrait se résumer ainsi: Attribuer une classe à un objet à partir d'une description textuelle et d'une image. On pourrait imaginer toutes sortes de déclinaisons:

* Commerciales
  * Classification automatique de produits mis en vente sur un site de e-commerce (le but original du challenge).
    * Assister un utilisateur dans le choix d'une catégorie lors de la mise en vente de son produit. Quand on connaît le grand nombre de catégories typiquement proposées sur les sites de e-commerce, on peut imaginer la confusion des utilisateurs.
    * On pourrait même imaginer un système plus coercitif qui impose une catégorie en fonction du contenu pour éviter les erreurs de catégorisation.
    * Fournir une recherche intelligente des produits qui puisse automatiquement traduire une description de produit saisie en classe de produits et ainsi produire des résultats pertinents.   
* Génériques
  * Un moteur de recherche d'objets à partir de description ou même d'images.
  * Moteur de recherche d'image à partir d'une description et réciproquement (si les modèles individuels de nlp et de computer vision sont suffisamment performant)



## Equipe

* Mathis Poignet: TODO
* Karim Hadjar: TODO
* Julien Noel du Payrat ([GitHub](https://github.com/surfncode) / [LinkedIn](https://www.linkedin.com/in/julien-noel-du-payrat-01854558))
  * J'ai un background de développeur depuis plus de 15 ans, en revanche, je fais encore mes armes en data science. Malgré une curiosité naturelle pour ce domaine, c'est quelque chose que j'ai découvert pendant la formation.


# Initialisation

## Montage du projet
Accès au dossier du projet sur Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd drive/MyDrive/Projet_Rakuten

## Initialisation des variables globales

In [ ]:
random_state = 123
# TODO: est-ce qu'on utilise ça ?
label2categorie = {
    10	:	"Livres",
    40	:	"Gaming",
    50	:	"Gaming",
    60	:	"Gaming",
    1140	:	"Jouets",
    1160	:	"Jouets",
    1180	:	"Jouets",
    1280	:	"Jouets",
    1281	:	"Jouets",
    1300	:	"Jouets",
    1301	:	"Bazar",
    1302	:	"Jouets",
    1320	:	"Equipement",
    1560	:	"Mobilier",
    1920	:	"Décoration",
    1940	:	"Bazar",
    2060	:	"Décoration",
    2220	:	"Equipement",
    2280	:	"Livres",
    2403	:	"Livres",
    2462	:	"Gaming",
    2522	:	"Livres",
    2582	:	"Mobilier",
    2583	:	"Equipement",
    2585	:	"Equipement",
    2705	:	"Livres",
    2905	:	"Gaming",
}



## Importation des librairies

In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm import tqdm
import html

from langdetect import detect_langs
# Force langdetect to have deterministic results
from langdetect import DetectorFactory
DetectorFactory.seed = random_state

# Télécharger et utiliser les stopwords français (STOPWORDS pour l'anglais)
from wordcloud import WordCloud #, STOPWORDS
from nltk.corpus import stopwords as nltk_stopwords
import re
import nltk
nltk.download('stopwords')

from collections import Counter




## Chargement des données

Chargeons X_train.csv et affichons un premier aperçu des données

In [ ]:
X = pd.read_csv('Datasets/X_train.csv', index_col=0)
X.head(10)

In [ ]:
X.info()

Ce premier coup d'œil nous montre que comme il fallait s'y attendre pour un champ optionnel, le champ **description** contient un grand nombre de valeurs nulles. Nous reviendrons là dessus...
Les autres colonnes ne contiennent pas de valeurs nulles.

Chargeons maintenant Y_train.csv

In [ ]:
Y = pd.read_csv("Datasets/Y_train.csv",index_col=0)
display(Y.head(10))

display(Y.info())

On voit qu'il n'y a pas de valeurs manquantes et que les deux fichiers **X_train.csv**, **Y_train.csv** ont bien le même nb d'entrées.

On va maintenant réunir les variables explicatives et la variable cible dans un même DataFrame **df** pour faciliter nos explorations. Au passage ça nous permettra de vérifier qu'on a bien une correspondance totale entre les deux indexes.

In [ ]:
df = X.join(Y,how='inner')
if df.shape[0] == X.shape[0] or df.shape[0] == Y.shape[0]:
  print("""
  Tous les fichiers ainsi que leur inner merge ont bien le même nombre
  d'entrées
  """)
display(df.head())

On constate que le merge s'est bien passé. On a maintenant un DataFrame **df** rattachant les variables de **X** et la colonne cible **prdtypecode**.



# Exploration des données

## Vérification des doublons

Vérifions que le dataset ne contient pas de doublons de lignes

In [ ]:
df.duplicated().sum()

Il n'y a pas de lignes dupliquées.

Vérifions maintenant les doublons pour chaque variable explicative séparément

In [ ]:
print("doublons de designation : ",df['designation'].duplicated().sum())
print("doublons de description : ",df['description'].duplicated().sum())
print("doublons de productid : ", df['productid'].duplicated().sum())
print("doublons de imageid : ", df['imageid'].duplicated().sum())

2651 doublons apparaissent pour **designation**. Même si le nombre n'est pas anodin, il pourrait s'expliquer par le des copié collé de la désignation au moment de mettre un produit en vente sur le site de Rakuten.

Le nombre de doublons de **description** est plus préoccupant, il semble vraiment élevé pour un champ avec des textes aussi longs. Assurons nous que les na n'ont pas été comptés comme doublons

In [ ]:
df[df['description'].isna() == False]["description"].duplicated().sum()

Après avoir éliminé les na, il ne reste plus que 7610 doublons. C'est déjà mieux.

Tentons maintenant de repérer combien parmi ces doublons de **designation** et **description** sont communs aux deux variables réunies.

In [ ]:
des_descr_duplicates = df[['designation','description']][df['description'].isna() == False].duplicated().sum()
des_descr_duplicates

1294 produits, bien qu'ayant des productid, ont des couples **designation**, **description** communs.

On peut donc obtenir maintenant le compte réel des doublons pour les colonnes traitées séparément.

In [ ]:
print("doublons des colonnes en enlevant les doublons communs")
print("designation : ",df['designation'].duplicated().sum() - des_descr_duplicates)
print("description : "
  ,df[df['description'].isna() == False]["description"].duplicated().sum() - des_descr_duplicates)

Le nombre de doublon est assez important et on ne peut pas les supprimer sans réfléchir. Il va falloir évaluer leur répartition en regard de la variable cible pour se décider

## Analyse de la variable cible (**prdtypecode**)

Commençons par compter le nombre de classes possibles que peut prendre la variable cible

In [ ]:
print("La variable cible compte %d classes possibles" % df["prdtypecode"].nunique())

Regardons maintenant le nombre de produits de chaque type

In [ ]:
fig = plt.figure(figsize = (16, 4))
sns.countplot(x = df['prdtypecode'], order = df['prdtypecode'].value_counts().index)
plt.title('Nombre de produits par type')
plt.show();

On constate que certaines classes (à droite du graphique) comme la classe **60** sont sous-représentées. Au contraire, la classe **2583** compte environ deux fois plus d'instances que les classes adjacentes à sa droite.

In [ ]:
fig = plt.figure(figsize = (16, 4))
# Identifier les doublons dans 'description' qui ne sont pas NaN
df_doublons = df[(df['description'].isna() == False) & (df[df['description'].isna() == False]["description"].duplicated(keep = False))]

# Compter et afficher le nombre de doublons par catégorie 'prdtypecode'
sns.countplot(x = df_doublons['prdtypecode'], order = df_doublons['prdtypecode'].value_counts().index)
plt.title('Nombre de doublons désignation / description par type')
plt.show();


print("\n Nombre de doublons par designation")
print(df_doublons['designation'].value_counts())
print("\n Nombre de doublons par description")
df_doublons['description'].value_counts()

La répartition des doublons est assez similaire à la répartition des produits par rapport à la variable cible.

On pourrait donc décider de supprimer les doublons sans casser la répartition initiale. Mais est-ce gênant de les garder? (TODO)

Les type codes ne sont pas très parlants si on veut se faire une idée du contenu des classes. Nous sommes parvenus à trouver sur (TODO: lien vers la source) les labels correspondants aux différents codes.

Pour la suite, nous allons construire un dictionnaire nous permettant d'associer les codes à leurs labels respectifs afin que les visuels gagnent en clarté.

In [ ]:
prdcodetype2label = {
    10 : "Livre occasion",
    40 : "Jeu vidéo, accessoire tech.",
    50 : "Accessoire Console",
    60 : "Console de jeu",
    1140 : "Figurine",
    1160 : "Carte Collection",
    1180 : "Jeu Plateau",
    1280 : "Jouet enfant, déguisement",
    1281 : "Jeu de société",
    1300 : "Jouet tech",
    1301 : "Paire de chaussettes",
    1302 : "Jeu extérieur, vêtement",
    1320 : "Autour du bébé",
    1560 : "Mobilier intérieur",
    1920 : "Chambre",
    1940 : "Cuisine",
    2060 : "Décoration intérieure",
    2220 : "Animal",
    2280 : "Revues et journaux",
    2403 : "Magazines, livres et BDs",
    2462 : "Jeu occasion",
    2522 : "Bureautique et papeterie",
    2582 : "Mobilier extérieur",
    2583 : "Autour de la piscine",
    2585 : "Bricolage",
    2705 : "Livre neuf",
    2905 : "Jeu PC",
}

Ajoutons une colonne **categorie** à **df**. Celle ci va contenir le label associé au **prdtypecode**

In [ ]:
df['categorie'] = df['prdtypecode'].map(prdcodetype2label)
df.head()

## Valeurs nulles de **description**

Intéressons nous maintenant aux valeurs nulles de la variable **description** que nous avons remarqué lors du chargement des données.

Calculons déjà leur proportion par rapport aux données totales.

In [ ]:
print("Les nan de descriptions représentent %d%% des données" \
      % (df['description'].isna().sum() * 100 / df.shape[0]))

Commençons à réfléchir à que faire de ces nans. On peut déjà envisager plusieurs options:
* On pourrait simplement supprimer les lignes qui en contiennent même si ça représente tout de même une part importante des données. De plus, cela viendrait en quelque sorte à l'encontre de l'objectif du projet qui spécifie bien que la colonne **description** est optionnelle. On doit donc prendre ça en compte dans notre approche
* On pourrait les remplacer par des chaînes de caractères vides. Bien que cette possibilité semble simple, nous avons beaucoup d'incertitudes quant à l'impact que ça aurait sur les modèles de prédictions.
* Sachant que le champ **designation** n'est jamais vide, on pourrait ajouter une troisième variable qui soit la concaténation des champs **designation** et **description**. Ainsi, cette nouvelle variable contiendrait au moins le contenu du champ **designation** nous ôtant ce problème de nan.

A première vue, c'est la troisième option qui semble la plus judicieuse, toutefois, pour essayer d'y voir plus clair, affichons un graphique en barre qui permettent de voir la proportion de nan pour chaque type de produit

Maintenant que nous avons des labels, on peut s'en servir plutôt que d'afficher des codes.

In [ ]:
# Transforme la colonne categorie en string
df.categorie = df.categorie.astype(str)

# Calcul du nombre d'apparitions de chaque catégorie
category_counts = df['categorie'].value_counts()

# Calcul des pourcentages de NaN et non-NaN pour chaque catégorie
category_nan_counts = df.groupby('categorie')['description'].apply(lambda x: x.isna().sum())
category_non_nan_counts = df.groupby('categorie')['description'].apply(lambda x: x.notna().sum())


# Tri des catégories par nombre d'apparitions
sorted_categories = category_counts.index.tolist()

# Tri des pourcentages de NaN et non-NaN selon l'ordre des catégories
sorted_nan_counts = category_nan_counts.reindex(sorted_categories)
sorted_non_nan_counts = category_non_nan_counts.reindex(sorted_categories)

# Création du graphique en barres
fig = go.Figure()

# Ajout des barres pour les valeurs non-NaN
fig.add_trace(go.Bar(
    name='Non-NaN',
    x=sorted_categories,
    y=sorted_non_nan_counts,
    marker_color='green'
))

# Ajout des barres pour les valeurs NaN
fig.add_trace(go.Bar(
    name='NaN',
    x=sorted_categories,
    y=sorted_nan_counts,
    marker_color='red'
))

# Mise à jour des paramètres du layout
fig.update_layout(
    barmode='stack',
    title='Répartition des NaN dans la Description par Catégorie',
    xaxis_title='Catégorie',
    yaxis_title='Pourcentage'
)

fig.update_xaxes(tickangle=60)

fig.show()




Cela nous confirme que l'option 1 est inenvisageable. Supprimer les lignes dont la **description** est vide revient presque à supprimer certaines catégories.
On préférera l'option 3: concaténer les champs **designation** et **description**.

## Longueur des textes

Nous n'avons pas encore parlé du type de texte qu'on peut trouver dans **designation** et **description**.

Penchons-nous sur la longueur des textes qu'on peut y trouver.

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
sns.boxplot(df['designation'].str.len().values)
plt.xlabel("designation")
plt.xticks([])
plt.subplot(122)
# On pense à enlever les na de description pour ne pas fausser les stats
sns.boxplot(data=df['description'][df['description'].isna()==False].str.len().values)
plt.xlabel("description")
plt.xticks([])
plt.suptitle("Distribution des longueurs de texte (en nb de caractères)")
plt.show()

Comme il fallait s'y attendre, on constate que les longueurs de texte dans **description** sont largement supérieures à celles qu'on trouve dans **designation**.

Dans les deux cas, on note un grand nombre d'outliers au dessus de l'interval interquartile `q3 + (q3 - q1 * 1.5)`. La distribution des outliers pour **description** est particulièrement étendue vers le haut puisqu'elle monte jusqu'à plus de 12,000 caractères alors que la valeur la plus haute située dans la fourchette *typique* se situe vers 2000.

Un autre information importante apparaît. Il semble qu'il y ait un certain nombre de valeurs proches de zéro caractères. Pour tenter d'y voir plus clair affichons le nombre de d’occurrences de **description** ayant une longueur inférieure à 5.



In [ ]:
df[df['description'].str.len() < 5]['description'].shape[0]

Seules 70 **descriptions** ont une longueur inférieure à 5. Ce cas reste minoritaire.

## HTML dans les données textuelles

En parcourant les données tabulaires de **X_train.csv**, on s'aperçoit qu'un grand nombre de textes contiennent du HTML, soit sous forme de tags, soit sous forme d'entités. Par exemple, on trouve des **description** comme ceci:

In [ ]:
df.iloc[36]['description']

On constate la présence de plusieurs tags HTML dans le texte notamment `<b></b>` et `<p></p>`

En outre, le texte contient également des caractères encodés sous la forme d'entités html `&#xx;` où xx représente deux digits en base hexadécimal. Par exemple `&#39;` représente une apostrophe. D'autres exemples dans le dataset contiennent des entités au format: `&<symbol-name>;` comme `&amp;` qui représente le caractère `&`.

Tentons, d'en savoir plus sur la fréquence de **description** contenant du HTML et/ou des entités

In [ ]:
regex_tags = re.compile('<.*?>')
regex_entities = re.compile('&(?:#\d+|\w+);')

description_without_na = df['description'][df['description'].isna() == False]
has_entities = description_without_na.str.contains(regex_entities)
has_html = description_without_na.str.contains(regex_tags)

plt.figure(figsize=(10,5))
plt.suptitle("Proportion de descriptions avec des tags ou des entités HTML (hors NA)")

plt.subplot(121)
plt.pie([has_html.sum(),description_without_na.shape[0]]
        ,labels=['Avec tags','Sans tags']
       ,explode=[0.2,0]
       ,autopct = lambda p : ("%.2f%%" % p)
       ,pctdistance = 0.7
       ,labeldistance=1.2
       ,shadow=True)

plt.subplot(122)
plt.pie([has_entities.sum(),description_without_na.shape[0]]
        ,labels=['Avec entités','Sans entités']
       ,explode=[0.2,0]
       ,autopct = lambda p : ("%.2f%%" % p)
       ,pctdistance = 0.7
       ,labeldistance=1.2
       ,shadow=True)

plt.show()




Ici, on peut voir qu'une proportion significative de **descriptions** contient des tag ou entités HTML.

**designation** contient-il, lui aussi du HTML ? Tentons de répondre à cette question.

In [ ]:
has_html = df['designation'].str.contains(regex_tags)
has_html.sum()

Seule une **designation** semble contenir des tags HTML. Cela semble curieux. Examinons son contenu:

In [ ]:
print(df.iloc[has_html[has_html].index[0]]['designation'])

La ligne est assez longue mais on peut voir que ce qu'on prenait pour un tag HTML n'est est pas un: `<Att>`

Maintenant, regardons si *designation* contient des entités HTML.

In [ ]:
has_entities = df['designation'].str.contains(regex_entities)
has_entities.sum()

990 occurrences de **designation** contiennent des entités html. C'est relativement peu.

Nous désirons commencer à examiner les langues ainsi que la fréquence des mots. Les éléments HTML que nous avons trouvés risquent de nous gêner dans ces tâches. Pour nous faciliter la tâche nous allons les supprimer.

Créons une fonction qui va permettre de supprimer les tags HTML et remplacer les entités par leur caractères normaux.

In [ ]:


def clean_html_stuff(col):
  regex_tags_nl = re.compile('<\s*(?:br|p|li)\s?.*?>',flags=re.IGNORECASE)
  regex_tags_space = re.compile('<.*?>')

  new_col = []
  for text in tqdm(col):
    if not pd.isnull(text):
      # remplacement de certains tags par '\n'
      # Ca permet de conserver un peu de la structure du texte au cas
      # où les modèles de nlp et/ou deep learning y soit sensibles
      # (c'est aussi plus agréable à l'oeil)
      text = re.sub(regex_tags_nl,'\n',text)
      # remplacement des tags restants par des espaces
      # J'ai choisi un espace plutôt qu'une chaine vide
      # pour éviter de concaténer des mots par erreur.
      # Ex remplacement par chaine vide:
      # "info<span>importante</span>" => "infoimportante"
      # Ex remplacement par espace
      # "info<span>importante</span>" => "info importante "
      # Ca produit quelques espaces en plus mais ça ne devrait pas poser
      # de problèmes
      text = re.sub(regex_tags_space,' ',text)
      # J'ai choisi de faire le remplacement des entité html après celui des tags
      # pour eviter le scenario ci-dessous
      # Remplacement avant tags:
      # "Enceinte &lt; 5kg et &gt; 8DB" => "Enceinte 8DB"
      # Remplacement après tags:
      # "Enceinte &lt; 5kg et &gt; 8DB" => "Enceinte < 5kg et > 8DB"
      # L'inconvénient en revanche, c'est que quelques tag qui étaient
      # encodés avec des &lt; &gt; subsitent mais ils sont peu
      # nombreux sur le volume des données
      text = html.unescape(text)
    new_col.append(text)

  return new_col

Nous pouvons maintenant appliquer cette fonction aux variables **designation** et **description**

In [ ]:
df['designation'] = clean_html_stuff(df['designation'])
df['description'] = clean_html_stuff(df['description'])

## Analyse des langues

En parcourant les données, on peut constater qu'hormis du HTML les données textes semblent être dans plusieurs langues. Beaucoup semblent rédigées en français mais on trouve également de l'anglais et de l'allemand dans des proportions non négligeables. Il pourrait aussi y avoir d'autres langues que nous n'avons pas remarqué, le volume de données étant important.

Pour tenter d'y voir plus clair, on peut recourir à la librairie **langdetect** qui permet de détecter la langue la plus probable d'un texte. Selon la documentation, plus le texte est long, plus la fiabilité de la détection augmente. Commençons donc par créer ajouter une variable **text** qui sera la concaténation de **designation** et **description**

In [ ]:
df['description'] = df['description'].fillna('')
df['text'] = df['designation'] + " - " + df['description']

Nous pouvons maintenant procéder à la détection des langues sur cette la variable **text**. Nous allons ajouter deux colonnes: **lang** et **lang_prob** contenant respectivement le code à 2 lettres du langage (fr,en,de etc...) et la probabilité que cette langue ait été détectée correctement.

Comme cette détection est lente, nous sauvegardons son résultat dans un fichier csv **Output/data-exploration/lang.csv**. Ainsi, si ce fichier est présent, les executions suivantes de cette cellule n'auront pas à refaire la détection.

In [ ]:
if 'lang' in df.columns:
  df.drop(['lang'],axis=1,inplace=True)
if 'lang_prob' in df.columns:
  df.drop(['lang_prob'],axis=1,inplace=True)

output_dir = "Output/data-exploration"
lang_file_path = output_dir + "/lang.csv"
lang_file = Path(lang_file_path)
if lang_file.exists():
  print("chargement du fichier existant:",lang_file_path)
  lang_df = pd.read_csv(lang_file_path,index_col=0)

else :
  print("detection des langues")
  lang_text = []
  lang_text_prob = []
  for text in tqdm(df['text']):
    detected_langs = detect_langs(text)
    if len(detected_langs) > 0:
      lang_text.append(detected_langs[0].lang)
      lang_text_prob.append(detected_langs[0].prob)
    else:
      lang_text.append(np.NaN)
      lang_text_prob.append(np.NaN)

  lang_df = pd.DataFrame(index=df.index,columns=['lang','lang_prob'])
  lang_df['lang'] = lang_text
  lang_df['lang_prob'] = lang_text_prob
  print("sauvgarde dans le fichier:",lang_file_path)
  Path(output_dir).mkdir(parents=True, exist_ok=True)
  lang_df.to_csv(lang_file_path)



df = df.join(lang_df,how="inner")

Affichons maintenant **df** avec les nouvelles colonnes issues de la détection

In [ ]:
df.head()

On voit que la détection semble avoir fonctionné. A première vue, la fiabilité de la détection est excellente mais vérifions tout de même sur l'ensemble des données.

In [ ]:
df['lang_prob'].describe()

On observe que les probabilités de détection correctes sont en général très bonnes. Dès le premier quartile (25%), on a déjà une probabilité d'à peu près 0.99. La moyenne de 95% et la médiane de 99.9% nous confirme qu'on peut avoir confiance en la détection.

Le code de la détection peut produire des valeurs nulles si aucune langue n'a pu être détectée par **langdetect**, vérifions que ce n'est pas le cas.

In [ ]:
df[['lang','lang_prob']].info()


On voit que ni **lang**, ni **lang_prob**  ne contiennent de valeurs manquantes.

On peut supprimer la colonne **lang_prob** maintenant que nous sommes rassurés sur la qualité de la détection

In [ ]:
df.drop('lang_prob',axis=1,inplace=True)

Maintenant que nous avons une langue pour chaque observation, on peut examiner la répartition des contenus par langue. Observons ça sur un graphique en barres.

In [ ]:
# Calcul du nombre de valeurs
total = df['lang'].value_counts().sum()
# Calcul du cumul des valeurs
cumul = df['lang'].value_counts().cumsum() / total *100

# graphique en barres avec les comptes par langue
fig, ax1 = plt.subplots(figsize = (12, 4))
sns.countplot(x = df.lang, order = df.lang.value_counts().index, ax = ax1)

# Création d'un deuxième axe y pour le cumul
ax2 = ax1.twinx()
# Ligne de cumul sur le deuxième axe y
ax2.plot(cumul.index, cumul.values, color='red', marker='o', linestyle='-', linewidth=2)
ax2.set_ylabel('Cumul')
# Ajustement de l'échelle du deuxième axe
ax2.set_ylim(0, cumul.values[-1])

plt.title('Nombre de contenus par langues détectés')
# Afficher la grille seulement pour le deuxième axe y
ax2.grid(None)
ax1.grid(None)
ax1.yaxis.grid(None)
ax2.yaxis.grid(True)
# Ajout annotations
plt.text(cumul.index[2], cumul.values[2] - 10, str(round(cumul.values[2],1)), ha='center', va='bottom')
plt.text(cumul.index[4], cumul.values[4] - 10, str(round(cumul.values[4],1)), ha='center', va='bottom')
plt.text(cumul.index[6], cumul.values[6] - 10, str(round(cumul.values[6],1)), ha='center', va='bottom')
plt.show();

On observe que le dataset ne compte pas moins de 16 langues. Cependant la plupart sont extrêmement minoritaires. Trois langues sont largement majoritaires: Le français, l'anglais puis l'allemand. A elles seules, elles représentent plus de 95% du contenu.

La répartition parmi ces trois langues est également hétéroclite puisque le français compte déjà pour plus de 75% suivi de très loin par l'anglais qui compte pour 5 fois moins et enfin de l'allemand représentant à peu près 1/3 du contenu anglais.

Pour se faire une idée plus juste des proportions. Observons la répartition des langues sur un camembert. On considérera que les langues qui ne font pas partie d'une des trois plus fréquentes peuvent être reléguées dans une catégorie autre afin de ne pas nuire à la lisibilité du graphique.

In [ ]:
lang_simple = df['lang']
# On obtients la liste des langues minoritaires en excluant les trois
# majoritaires puis on les remplace par "other" pour ne pas surcharger le graph
other_langs = df['lang'].value_counts().index[3:]
lang_simple = lang_simple.replace(other_langs,"other")
lang_counts = lang_simple.value_counts()

# Créer le diagramme en camembert avec Plotly
fig = go.Figure(data=[go.Pie(labels=lang_counts.index, values=lang_counts.values)])
fig.show()

On retrouve d'une façon plus explicite les chiffres qu'on a vu sur le graphique en barres. Le français en tête avec 77% suivi de l'anglais comptant pour 15% puis de l'allemand avec 3%. Quant au cumul des autres langues, il atteint à peine 5%.

La plupart des modèles ne NLP fonctionnent avec l'anglais. Bien qu'il y ait des versions adaptées pour le français, ça reste des modèles différents. Admettons qu'on veuille résoudre ce problème de classification pour le français et l'anglais séparément en omettant les autres langues minoritaires, ça pourrait impliquer de doubler la charge de travail.

Une autre technique consisterait à ne conserver que le contenu français cependant ça représenterait tout de même une perte de données de 25% environ. Avant de considérer cette alternative, vérifions la répartition des langues par type de produit. On ne peut se permettre de supprimer une langue si celle-ci représente une part importante des observations pour un type de produit.  

In [ ]:


plt.figure(figsize=(12,12))
sns.displot(y=df['categorie'],hue=lang_simple,multiple="stack",aspect=1.7,binwidth=2)
plt.title("Repartition des langues par types de produit")
plt.show()

Un point rassurant est que le français constitue la majorité (relative et non absolue) dans chaque type de produit. Cependant on observe quand même que pour certains types comme *jeu de plateau* cette majorité relative est ténue. Retirer les autres langues alors que le nombre de produits y est déjà faible pourrait produire un déséquilibre dans la distribution des classes de produits et compromettre la fiabilité de la classification.

Une autre technique qui éviterait ces écueils consisterait à traduire toutes les langues vers le français. La faisabilité reste encore à expérimenter sur un dataset de cette taille mais ça vaudra la peine de s'y pencher pendant la phase de pre-processing

## Fréquence des mots sur **designation** et **text**

Une façon d'approfondir notre analyse des données consiste à observer la fréquence des mots contenus dans les variables textuelles.

Ajoutons deux colonnes **mots_designation** et **mots_text** qui vont contenir la liste des mots extraits depuis leur colonne respective **designation** et **text**. Étudier séparément chacune de ces colonnes devrait nous permettre de souligner les différences entre la fréquence globale des mots et celle de **designation**.

On procède à l'extraction des mots par une *tokenization maison* à base d'expressions régulières tout en étant vigilant à exclure les mots qui apportent peu d'informations à notre analyse fréquentielle.  

In [ ]:
# Création d'une liste personnalisée de stopwords français, anglais et allemends
french_stopwords = set(nltk_stopwords.words('french'))
english_stopwords = set(nltk_stopwords.words('english'))
dutch_stopwords = set(nltk_stopwords.words('dutch'))
combined_stopwords = french_stopwords.union(english_stopwords)

# Fonction pour nettoyer et extraire les mots, en excluant les stopwords
def extract_words(text):
    # Utilisation d'une expression régulière pour ne conserver que les mots
    words = re.findall(r'\w+', text.lower())
    # Filtrage des mots en excluant les stopwords, les chiffres et les mots de moins de 3 lettres, et suppression des doublons
    unique_words = {word for word in words if word not in combined_stopwords and not word.isdigit() and len(word) > 2}
    return list(unique_words)

# Ajout colonne mots_designation contenant les mots de la colonne designation
df['mots_designation'] = df['designation'].apply(extract_words)
# Ajout colonne text contenant les mots de la colonne description
df['mots_text'] = df['text'].apply(extract_words)

# Affichage du résultat
display(df.head())
display(df['text'][0])
df['mots_text'][0]

L'extraction s'est bien déroulée, on constate l'apparition des deux nouvelles colonnes **mots_designation** et **mots_text**.

On peut désormais afficher un premier nuage de mots qui représente la fréquence des mots contenus dans **designation**

In [ ]:
# Génération du nuage de mots
# Création de la chaine all_word_designation en combinant toutes les listes de mots
all_word_designation = ' '.join([word for words_list in df['mots_designation'] for word in words_list])

wordcloud_designation = WordCloud(width = 800, height = 800, max_words = 1000, background_color = 'black',
                      colormap = "nipy_spectral").generate(all_word_designation)

plt.imshow(wordcloud_designation)
plt.axis("off")
plt.title('Mots contenus dans la designation')
plt.tight_layout(pad=0)

On observe une nette prédominance des mots français en accord avec nos observations sur la répartition des langues. On a peine à distinguer quelques mots d'anglais en arrière-plan.

Quelques mots se distinguent clairement: lot, noir, piscine. A ce stade on peut s'interroger sur une possible correspondance entre fréquence d'un mot et appartenance à une catégorie de produit. Nous aurons l'occasion de creuser la question lorsqu'on affichera les fréquences de mots par catégories.

Examinons maintenant la fréquence globale des mots, c'est à dire la fréquence des mots contenus dans la variable **text**, qui pour rappel est issue de la concaténation de **designation** et **description**

In [ ]:
# Génération du nuage de mots
# Création de la chaine all_word_texte en combinant toutes les listes de mots
all_word_texte = ' '.join([word for words_list in df['mots_text'] for word in words_list])

wordcloud_texte = WordCloud(width = 800, height = 800, max_words = 1000, background_color = 'black',
                      colormap = "nipy_spectral").generate(all_word_texte)

plt.imshow(wordcloud_texte)
plt.axis("off")
plt.title('Mots contenus dans le texte designation + description')
plt.tight_layout(pad=0)

La prédominance du français est encore plus nette. Les rares mots anglais qu'on pouvait encore distinguer sont probablement encore plus en arrière-plan. On constate aussi que hormis *lot*, les mots les plus identifiables ont changé. C'est plutôt rassurant car ça signifie que le champ **description** apporte une information qui n'est pas redondante par rapport au champ **designation**.

Plusieurs mots en évidence ont une consonance plutôt marketing que descriptive. Notamment *neuf*, *haute*, *qualité*. C'est attendu dans du texte destiné à inciter à la vente de produits.

## Fréquence des mots dans le texte pour chaque catégorie

Penchons nous maintenant sur la fréquence des mots au sein de chaque catégorie. On voudrait afficher pour chaque catégorie la fréquence des 10 mots les plus courants.

Pour cela, on crée un DataFrame **df_plot** qui contient trois colonnes: **categorie**, **word** et **count**. Ce DataFrame aura donc 10 lignes par catégorie, chaque ligne correspondant à un des 10 mots les plus fréquents.

In [ ]:
# Attention 2/3 min de calcul si effectué sur mots_texte
# Calculer la fréquence des mots pour chaque catégorie
word_counts_by_category = {}
for category in df['categorie'].unique():
    # Filtrer les lignes par catégorie
    filtered_df = df[df['categorie'] == category]
    # Combiner tous les mots de cette catégorie
    all_words = sum(filtered_df['mots_text'], [])
    # Compter les mots et stocker dans le dictionnaire
    word_counts_by_category[category] = Counter(all_words)

In [ ]:
# Préparation des données pour Plotly
data = []
for category, counter in word_counts_by_category.items():
    for word, count in counter.most_common(10):  # Top 10 mots pour chaque catégorie
        data.append({'categorie': category, 'word': word, 'count': count})

df_plot = pd.DataFrame(data)
df_plot.head()

On peut maintenant utiliser **df_plot** pour afficher un ensemble de graphique en barre (un par catégorie) qui vont représenter les occurrences des dix mots les plus courants dans leur catégorie respective

In [ ]:
# Obtenir les catégories uniques
categories = df_plot['categorie'].unique()

# Créer un ensemble de subplots (3 colonnes, 9 lignes)
fig = make_subplots(rows=9, cols=3, subplot_titles=[f'Catégorie: {category}' for category in categories])

# Fonction pour ajouter des barres horizontales pour chaque catégorie
def add_bars(fig, df, category, row, col):
    filtered_df = df[df['categorie'] == category].sort_values(by='count')
    fig.add_trace(
        go.Bar(y=filtered_df['word'], x=filtered_df['count'], name=str(category), orientation='h'),
        row=row, col=col
    )

# Ajouter des barres pour chaque catégorie dans son subplot respectif
row = 1
col = 1
for category in categories:
    add_bars(fig, df_plot, category, row, col)
    col += 1
    if col > 3:
        col = 1
        row += 1

# Mise à jour de la mise en page
fig.update_layout(height=3000, width=1200, title_text="Fréquence des mots par catégorie de produit", showlegend=False)
fig.show()

Bien que la plupart des catégories aient des mots qui reflètent leur thèmes, on constate fréquemment que des mots génériques tel que *peut*, *plus*, *être*, *cette* apparaissent dans le top 10. Ces mots ont probablement échappé à la liste des stop words censée les filtrer. Il n'apportent pas d'information sur la catégorie.

D'autres termes typiques d'un vocabulaire produit sont notables tels que *dimension*, *taille*, *longueur*.  Eux non plus ne devraient pas aider à la classification.

Enfin, on retrouve les mots classiques du marketing de vente qu'on avait déjà repéré: *haute*, *qualité*, *facile*.

Si comme on le suppose, ces mots ne contribuent pas à la variance expliquée entre les classes de produits, il devraient être filtrés naturellement lors de la phase de sélection de features d'une réduction de dimension efficace.

On note aussi certaines catégories particulières comme **livre neuf** dont les mots ne reflètent pas la nature. On dirait plutôt que les mots qui ressortent sont ceux des titres les plus vendus. Curieusement la catégorie **livre occasion** de partage pas cette caractéristique puisque son vocable correspond bien au monde de l'édition

Affichons maintenant les nuages de mots par catégorie. Cela nous fournira une vision plus exhaustive qu'un top 10.

In [ ]:
#TODO: ameliorer tqdm qui n'affiche pas le nombre totale d'iterations

# Création de subplots pour les nuages de mots (3 colonnes x 9 lignes)
fig, axes = plt.subplots(9, 3, figsize=(15, 45))

# Assurer que les axes sont aplatis en un seul tableau si nécessaire
axes = axes.flatten()

for i, (category, counts) in tqdm( enumerate(word_counts_by_category.items()) ):
    # Génération du nuage de mots pour la catégorie
    wordcloud = WordCloud(width=800, height=800, max_words=1000, background_color='black', colormap="nipy_spectral").generate_from_frequencies(counts)

    # Affichage du nuage de mots dans le subplot correspondant
    axes[i].imshow(wordcloud, interpolation='bilinear')
    axes[i].axis("off")
    axes[i].set_title(f'Catégorie: {category}')

# Cacher les axes supplémentaires s'il y en a
for j in range(i + 1, 27):
    axes[j].axis('off')

# Ajustement de la mise en page
plt.tight_layout(pad=0)
plt.show()

Bien sûr les mots qu'on avait repéré dans le top 10 sont ici mis en évidence. Cette représentation permet toutefois de repérer certains mots plus en arrière plan qui n'apparaissaient pas sur le graphique précédent.

Pour reprendre notre exemple de **livre neuf**, on observe la présence en retrait de quelques mots plus proches de l'édition tels que: *livre*, *ouvrage*, *tome*.

Ça souligne une fois de plus l'importance qu'aura un bon process de sélection de features qui devra écarter les features les plus évidentes au profit de de feature plus explicatives.

## test wordCloud avec masque

In [ ]:
# Génération du nuage de mots
image = plt.imread('Assets/Images/SuperNES.jpg')

categorie_specifique = 'Console de jeu'

if categorie_specifique in word_counts_by_category:
    # Génération du nuage de mots pour la catégorie
    wordcloud = WordCloud(width=800, height=800, max_words=1000,mask = image, background_color='black',
                          colormap="nipy_spectral").generate_from_frequencies(word_counts_by_category[categorie_specifique])

    # Affichage du nuage de mots
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f'Catégorie "{categorie_specifique}"')
    plt.show()

In [ ]:
# Génération du nuage de mots
image = plt.imread('Assets/Images/manette_xbox.jpg')

categorie_specifique = 'Jeu occasion'

if categorie_specifique in word_counts_by_category:
    # Génération du nuage de mots pour la catégorie
    wordcloud = WordCloud(width=800, height=800, max_words=1000,mask = image, background_color='#E0E0E0',
                          colormap="nipy_spectral").generate_from_frequencies(word_counts_by_category[categorie_specifique])

    # Affichage du nuage de mots
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f'Catégorie "{categorie_specifique}"')
    plt.show()

## Les images

TODO: parler du nb d'images dans les dossiers qui correspond bien au nb de produits dans X_train.csv

TODO: parcourir les dossiers en python et vérifier que chaque image est bien présente ?

TODO: parler des images miniatures dans un rectangle blanc. Montrer des exemples

TODO: pourcentage de blanc dans les images ?

Ajout d'un champs contenant le nom du fichier image.
TODO: necessaire pour le traitement des images ?

In [ ]:
# Conversion des colonnes 'imageid' et 'productid' en chaînes de caractères
df['imageid'] = df['imageid'].astype(str)
df['productid'] = df['productid'].astype(str)

#Ajout colonne nom_fichier
df['nom_fichier'] = "image_" + df['imageid'] + "_product_" + df['productid'] + ".jpg"

df.head()

# TODO: corriger l'orthographe

# TODO: sortir des chiffres (stats) pour compléter l'interpretation des graphs

# TODO: choisir une langue unique pour le code (anglais?)